<a href="https://colab.research.google.com/github/LM1997610/Networking-for-Big-Data/blob/main/NBD_HW2_scraps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!gdown 19baCZ9_wVNSBWVRJFtDIA4haBxv6IXAw

Downloading...
From: https://drive.google.com/uc?id=19baCZ9_wVNSBWVRJFtDIA4haBxv6IXAw
To: /content/Cell_a.zip
100% 43.3M/43.3M [00:00<00:00, 155MB/s]


In [ ]:
data = pd.read_csv('Cell_a.zip')

In [ ]:
class LWLDispatcher:
  def __init__(self, N, data):
    self.N = N
    self.clock = int(data.iloc[0,2])
    self.controller = DispatcherControl(N)
    self.job_controller = JobControl(data).register
    self.servers = [FCFS(id) for id in range(0,N)]
  

  def dispatch(self, task, µ = 0.1):
    new_job_id = str(int(task["Job_ID"]))+ "_" + str(int(task["Task_ID"]))
    arrival_time_task = int(task["Arrival_Time"])
    running_time_task = (task["CPU"]/µ)*1000     

    memory = task["Memory"]
    workload = np.array([sum(list(self.controller.details[s].values()))
                for s in range(0,self.N)])
    pointed = np.argmin(workload)
    check = False
    while check == False:
      if self.controller.memory_constraint(pointed, memory) == False:
        workload[pointed] = 1.1
        pointed = np.argmin(workload)
      else:
        check = True

    self.servers[pointed].add_to_queue(task)
    
    self.controller.memory_status[pointed] -= memory
    self.controller.details[pointed][new_job_id] = running_time_task
    if self.controller.status[pointed] == 0:
      self.controller.status[pointed] = 1
    self.controller.queue_length[pointed] += 1

  def time_manager(self, task):
    timestamp = int(task['Arrival_Time'])
    if self.clock != timestamp:
      for i in [k for k in range(self.N) if len(self.servers[k].queue)>0]:
        info = self.servers[i].queue
        temp = timestamp
        pre = info[0]['arrival_time']
        delta = temp - pre
        loop = True
        while loop:
          job = info[0]['id'].split('_')[0]
          d = info[0]['service_time'] - delta
          if d > 0:
            for j in range(1, len(self.servers[i].queue)):
              info[j]['waiting_time'] += delta
              loop = False
          else:
            self.servers[i].queue.pop(0)
            accomp = timestamp + d
            d = info[0]['service_time'] - delta
            self.job_controller[job]['Ntasks'] -= 1
            if accomp > self.job_controller[job]['completion']:
              self.job_controller[job]['completion'] = accomp
              d = delta

    self.clock = timestamp
            
class FCFS:
  def __init__(self, id):
    self.server_id = id 
    self.status = 0
    self.memory = 1 
    self.queue = []  
    self.queue_status = []  
    self.workload = 0 
    #self.list_done_task = []   # for messages with dispatcher 
    
  def dequeueing(self, task):
    self.queue.pop(0)           #send a message to dispatcher with timestamp info

  def add_to_queue(self, dizionario : dict, µ = 0.1 ):
    new_job_id = str(int(dizionario["Job_ID"]))+ "_" + str(int(dizionario["Task_ID"]))
    arrival_time_task = int(dizionario["Arrival_Time"])
    running_time_task = (dizionario["CPU"]/ µ)*1000     # from seconds to milliseconds
    a_dict = {"id": new_job_id, "arrival_time": arrival_time_task, "service_time" : running_time_task, 'waiting_time':0}
    if self.memory > dizionario["Memory"]: 
      self.memory -= dizionario["Memory"]
      self.queue.append(a_dict)   ### and do some work     
    else: 
      return 0

class JobControl:
  def __init__(self, df):
    arrival_times = np.array(pd.unique(df['Arrival_Time']))
    id_s = np.array(pd.unique(df['Job_ID']))
    count = np.array(df.groupby('Job_ID')["Task_ID"].count())
    self.register = {id:{'Ntasks':k, 'arrival':t, 'completion':0} for (id, k, t) in zip(id_s, count, arrival_times)}

class DispatcherControl:

  def __init__(self, N):
    self.servers_number = N
    self.status = {s:0 for s in range(0,N)}
    self.queue_length = {s:0 for s in range(0,N)}
    self.memory_status = {s:1 for s in range(0,N)}
    self.details = {s:{} for s in range(0,N)}

  def is_idle(self, server): 
    return self.status[server] == 0

  def memory_constraint(self, server, task_memory):
    return self.memory_status[server] - task_memory > 0

In [ ]:
a = LWLDispatcher(64, data)

In [ ]:
for q in range(20):
  task = data.iloc[q,].to_dict()
  a.time_manager(task)
  a.dispatch(task)

IndexError: ignored

In [ ]:
[k for k in range(a.N) if len(a.servers[k].queue)>0]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [ ]:
a.servers[2].queue

[{'id': '375000670586_0',
  'arrival_time': 608994453,
  'service_time': 241.75643920898398,
  'waiting_time': 0}]

In [ ]:
timestamp = data.iloc[0,2]
for i in range(1, data.shape[0]):
  temp = data.iloc[i,2]
  delta = timestamp - temp
  timestamp = temp 

KeyboardInterrupt: ignored